In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn 
import plotly.express as px

# Домашняя работа: деревья решений

В этой домашней работе вам предстоит научиться предсказывать цены товаров из маркетплейса Azamon.

Требования к домашней работе:
- Во всех графиках должны быть подписи через title, legend, etc.
- Во время обучения моделей проверяйте, что у вас не текут данные. Обычно это позитивно влияет на качество модели на тесте, но негативно влияет на оценку 🌚
- Если вы сдаете работу в Google Colaboratory, убедитесь, что ваша тетрадка доступна по ссылке.
- Использование мемов допускается, но необходимо соблюдать меру. Несодержательная работа, состоящая только из мемов, получает 0 баллов.

# Загрузка и подготовка данных

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/amazon_co-ecommerce_sample.csv').drop(columns=[
    'product_name',
    'index',
    'uniq_id',
    'customers_who_bought_this_item_also_bought',
    'items_customers_buy_after_viewing_this_item',
    'sellers',
    'description', # text
    'product_information', # text
    'product_description', # text
    'customer_questions_and_answers', # text
    'customer_reviews', # text
])

Mounted at /content/drive


## Очистка данных (1 балл)

Посмотрите на признаки. Есть ли в них пропуски? Какое соотношение между NaN'ами и общим количеством данных? Есть ли смысл выкидывать какие-либо данные из этого датасета?

In [3]:
#Число NaN в каждой категории
df.isna().sum()

manufacturer                           7
price                               1435
number_available_in_stock           2500
number_of_reviews                     18
number_of_answered_questions         765
average_review_rating                 18
amazon_category_and_sub_category     690
dtype: int64

In [4]:
#Процент NaN в каждой категории
df.isna().sum()/len(df) * 100

manufacturer                         0.07
price                               14.35
number_available_in_stock           25.00
number_of_reviews                    0.18
number_of_answered_questions         7.65
average_review_rating                0.18
amazon_category_and_sub_category     6.90
dtype: float64

Насколько я вижу, есть серьёзные потери числовых данных в категориях "цена" и "число доступных на рынке". Заполнять усреднёнными либо медианными данными четверть датафрейма опасно на предмет искажения данных по сравнению с реальностью. Попробуем прочистить датафрейм.

In [5]:
df_cut = df.dropna()

In [16]:
df_cut.count()

manufacturer                        5486
price                               5486
number_available_in_stock           5486
number_of_reviews                   5486
number_of_answered_questions        5486
average_review_rating               5486
amazon_category_and_sub_category    5486
dtype: int64

Многовато убрали. Придётся заполнять.

## Подготовка данных (3 балла)

Обработайте признаки. Выполните кодирование категориальных признаков, заполните пропуски в числовых признаках. Обратите внимание, что в датасете есть признак, который разбивается на несколько подпризнаков. Что это за признак? Закодируйте и его.

Дополнительные вопросы (+ 1 балл):
- Какие из признаков в этом датасете лучше кодировать через ordinal encoding?
- Какие из признаков допустимо кодировать через one-hot?

Прим.: суммарно за эту секцию можно получить до 4 баллов.

Насколько я понял, в этом датасете лучше кодировать через ordinal encoding amazon_category_and_sub_category и в теории возможно применить к number_available_in_stock из-за того, что они построены иерархическим образом.

In [7]:
df_copy = df

In [8]:
#Сохранение (не работает)
df = df_copy

In [9]:
df['price'] = df['price'].str.replace('£', '').str.replace(',', '') #убираем "мусорные" символы
df['price'] = df['price'].apply(lambda x: np.mean([float(i) for i in x.split('-')]) if isinstance(x, str) and '-' in x else float(x))
df['price'] = df['price'].fillna(df['price'].median())

In [10]:
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce')
df['number_of_reviews'] = df['number_of_reviews'].fillna(df['number_of_reviews'].mean())

Признак number_available_in_stock можно разбить на подпризнаки used, new и collectible и закодировать с помощью иерархического ordinal encoding, но подавляющее большинство строк имеет признак new, так что я просто заполнил пропуски.

In [11]:
#Утеряно 25% данных. Значения могут быть грубыми.
df['number_available_in_stock'] = df['number_available_in_stock'].str.extract('(\d+\.?\d*)', expand=False) #извлекаем числа
df['number_available_in_stock'] = pd.to_numeric(df['number_available_in_stock'], errors='coerce') 
mean_value = df['number_available_in_stock'].mean()
df['number_available_in_stock'] = df['number_available_in_stock'].fillna(mean_value)

Категорию также лучше 'amazon_category_and_sub_category' лучше всего закодировать через ordinal или one-hot encoding

In [12]:
#Пока у меня не получилось


In [13]:
df.count()

manufacturer                         9993
price                               10000
number_available_in_stock           10000
number_of_reviews                   10000
number_of_answered_questions         9235
average_review_rating                9982
amazon_category_and_sub_category     9310
dtype: int64

Осталось приемлимое количество строк, можно вырезать остальное (временная мера)

In [14]:
df.dropna(inplace=True)
df.count()

manufacturer                        8581
price                               8581
number_available_in_stock           8581
number_of_reviews                   8581
number_of_answered_questions        8581
average_review_rating               8581
amazon_category_and_sub_category    8581
dtype: int64

# Обучение модели (3 балла)

## Бейзлайн

Обучите базовую модель. Для этого используйте `sklearn.dummy.DummyRegressor`. Какое качество она показывает на тесте? Посчитайте MSE, RMSE.

## Дерево решений

Обучите регрессионное дерево решений, проверьте качество этой модели на тестовой выборке. Улучшилось ли качество по сравнению с базовой моделью? Оцените r2_score обученной модели.

## Линейная регрессия

Попробуйте обучить линейную регрессию с параметрами по умолчанию. Оцените r2_score на тестовой выборке. Сравните качество с деревом решений. 

# Гиперпараметры (2 балла)

Переберите несколько гиперпараметров (не более двух-трёх). Обратите внимание, как эти параметры влияют на ошибку модели на тестовой выборке. Постройте для глубины дерева график переобучения (fitting curve) аналогичный тому, что мы строили на занятии. Найдите глубину дерева, начиная с которой модель начинает переобучаться.

# Простое ансамблирование (1 балл)

В этой секции мы реализуем простой ансамбль деревьев.

In [15]:
class EnsembleTreeRegressor:
    def __init__(self, num_trees=5, samples_frac=0.8, **model_kwargs):
        self.num_trees= num_trees
        self._samples_frac = 0.8
        self._trees = [DecisionTreeRegressor(**model_kwargs) for _ in range(num_trees)]
    def fit(self, x, y: pd.Series):
        x = pd.DataFrame(x)
        y = y.reset_index(drop=True)
        for tree in self._trees:
            tree_x = x.sample(frac=self._samples_frac, random_state=42)
            tree_y = y[tree_x.index]
            tree.fit(tree_x, tree_y)
        return self

    def predict(self, x: pd.DataFrame):
        x = pd.DataFrame(x)
        res = []
        for i in range(self.num_trees):
          res.append(self._trees[i].predict(x))
        return sum(res) / len(res)

Проверьте, работает ли этот ансамбль лучше обычного дерева с параметрами по умолчанию?

Дополнительно переберите максимальную глубину дерева. Проверьте, насколько отличается момент начала переобучения у одиночного дерева и у ансамбля. Зависит ли этот момент от числа деревьев (`num_trees`)? От числа примеров для каждого дерева (`samples_frac`)? Постройте график fitting curve.